### Imports 

In [1]:
import os 
import sys
import pathlib
import requests
from io import StringIO, BytesIO
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

In [3]:
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 

In [4]:
import seaborn as sns

In [5]:
import numpy as np
import pandas as pd

In [6]:
from dateparser import parse

In [7]:
headers = {'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"}

### Some parameters 

### number of months to keep (last 12)

In [8]:
n_months = 24 # If you are running this in January, use this n_months = 24
lag = 0 # lag to realtime

In [9]:
current_year = parse('this year')
current_year = current_year.strftime("%Y")

In [10]:
year = int(current_year)

In [11]:
this_month = parse(f'{lag+1} months ago')
this_month = this_month.strftime("%m")

In [12]:
month = int(this_month)

In [13]:
enso_tracker_path = '../template'

In [14]:
figures_path = './figures'

In [15]:
if not pathlib.Path(figures_path).exists(): 
    pathlib.Path(figures_path).mkdir(parents=True)

In [16]:
last_valid = datetime(year, month, 1)

In [17]:
last_valid = last_valid + pd.offsets.MonthEnd(0)

# If you are running this in January, use this
# last_valid = last_valid + pd.offsets.MonthEnd(-12)

In [18]:
last_valid

Timestamp('2025-07-31 00:00:00')

In [19]:
start_date = last_valid - relativedelta(months=n_months)

# If you are running this in January, use this
# n_months = 12
# start_date = last_valid - relativedelta(months=n_months)

In [20]:
start_date

Timestamp('2023-07-31 00:00:00')

### Access and read the NINO3.4 index from the BoM 

In [21]:
import requests
from io import StringIO
import pandas as pd

def get_NINO34_anomaly(base_url="https://www.cpc.ncep.noaa.gov/data/indices/ersst5.nino.mth.91-20.ascii", start_date=None, end_date=None):
    """
    Get the NINO 3.4 anomaly data from the specified URL
    """
    
    # Make the request to get the data
    r = requests.get(base_url)

    if r.status_code != 200:
        print(f"Something went wrong with accessing the URL {base_url}")
        return None

    # Read the response content as a string
    data = r.text

    # Skip the first few lines until the header row
    data_lines = data.split('\n')[1:]

    # Read the data into a DataFrame
    nino_data = pd.read_csv(StringIO('\n'.join(data_lines)), sep='\s+', header=None, usecols=[0, 1, 9], names=['YR', 'MON', 'NINO3.4'])

    # Combine 'YR' and 'MON' columns into a datetime index
    nino_data['Date'] = pd.to_datetime(nino_data['YR'].astype(str) + '-' + nino_data['MON'].astype(str), format='%Y-%m')
    nino_data.set_index('Date', inplace=True)
    nino_data.drop(columns=['YR', 'MON'], inplace=True)

    # Filter by start and end dates if provided
    if start_date is not None:
        nino_data = nino_data.loc[start_date:]
    if end_date is not None:
        nino_data = nino_data.loc[:end_date]

    return nino_data


In [22]:
def get_LP_SOI(url='https://data.longpaddock.qld.gov.au/SeasonalClimateOutlook/SouthernOscillationIndex/SOIDataFiles/DailySOI1933-1992Base.txt', start_date=None, end_date=None): 
    """
    get the SOI from LongPaddock
    """
    
    from datetime import datetime, timedelta
    from dateutil.relativedelta import relativedelta
    import numpy as np 
    import pandas as pd
    
    r = requests.get(url)

    if r.status_code != 200: 
        print(f"something went wrong with accessing the URL {url_soi}")
    else: 
        soi_index = pd.read_table(BytesIO(r.content), sep='\s+', engine='python') 
        
    index = [datetime(int(year),1,1) + timedelta(int(day)-1) for year, day in soi_index.loc[:,['Year','Day']].values]
    soi_index.index = index
    soi_index = soi_index.resample('ME').mean()

    if not (start_date == end_date == None):
        soi_index = soi_index.loc[start_date:end_date,:]
    else: 
        start_date = (datetime.utcnow() - timedelta(15)) - relativedelta(months=n_months)
        soi_index = soi_index.loc[start_date:,:]
    
    return soi_index

In [23]:
def read_ENSO_categories(dpath = '../template', fname = 'Pacific_RCC_ENSO_tracker.csv'): 
    
    import pathlib
    import numpy as np 
    import pandas as pd 
    
    filename = pathlib.Path(dpath).joinpath(fname)
    
    enso_cat = pd.read_csv(filename)
    
    # goes from string to categories 
    cat_to_int = {} 

    cat_to_int['La_Nina'] = -3
    cat_to_int['La_Nina_alert'] = -2
    cat_to_int['La_Nina_watch'] = -1
    cat_to_int['neutral'] = 0
    cat_to_int['El_Nino_watch'] = 1 
    cat_to_int['El_Nino_alert'] = 2 
    cat_to_int['El_Nino'] = 3
    
    enso_cat_int = enso_cat.replace(cat_to_int)
    
    enso_cat_int.loc[:,'date'] = np.nan
    
    # build the index 
    enso_cat_int.loc[:,'date'] = enso_cat_int.loc[:,'YEAR'].astype(str) + "-" + enso_cat_int.loc[:,'MONTH'].astype(str).str.zfill(2)
    
    enso_cat_int = enso_cat_int.set_index('date').drop(['YEAR','MONTH'],axis=1) 
    
    enso_cat_int.index = pd.to_datetime(enso_cat_int.index) + pd.offsets.MonthEnd(0) 
    
    return enso_cat_int

### data munging

In [24]:
nino_index = get_NINO34_anomaly(start_date=start_date, end_date=last_valid)

In [25]:
nino_index = nino_index.reset_index()

In [26]:
nino_index.rename(columns={nino_index.columns[0]: ''}, inplace=True)

In [27]:
nino_index.set_index('', inplace=True)

In [28]:
nino_index.rename(columns={'NINO3.4': 'SST'}, inplace=True)

In [29]:
nino_index.index = nino_index.index.to_period('M').to_timestamp('M') + pd.offsets.MonthEnd(0)

In [30]:
nino_index

,SST
,
2023-08-31,1.35
2023-09-30,1.60
2023-10-31,1.72
2023-11-30,2.02
2023-12-31,2.03
2024-01-31,1.81
2024-02-29,1.52
2024-03-31,1.13
2024-04-30,0.77


In [31]:
soi_index = pd.read_csv('./SOI_NOAA.csv', index_col=0, parse_dates=True)

In [32]:
soi_index = soi_index.loc[start_date:last_valid,:]

In [33]:
# soi_index = get_LP_SOI(start_date=start_date, end_date=last_valid)

In [34]:
# soi_index = soi_index.loc[:,['SOI']]

In [35]:
# soi_index = soi_index.iloc[1:]

In [36]:
soi_index.index = soi_index.index + pd.offsets.MonthEnd(0)

### check 

In [37]:
if len(soi_index) != len(nino_index): 
    print(f"check the SOI and NINO3.4 index data, the SOI Index has length {len(soi_index)} and the NINO Index has length {len(nino_index)}")

### merge the NINO and SOI index 

In [38]:
ENSO_index = pd.concat([nino_index, soi_index], axis=1)

In [39]:
ENSO_index

,SST,SOI
2023-08-31,1.35,-1.013194
2023-09-30,1.60,-1.321736
2023-10-31,1.72,-0.494559
2023-11-30,2.02,-1.147119
2023-12-31,2.03,-0.384429
2024-01-31,1.81,0.258809
2024-02-29,1.52,-1.098070
2024-03-31,1.13,0.117237
2024-04-30,0.77,-0.245499
2024-05-31,0.23,0.896108


In [40]:
ENSO_index = ENSO_index.dropna()

In [41]:
ENSO_index.head()

,SST,SOI
2023-08-31,1.35,-1.013194
2023-09-30,1.60,-1.321736
2023-10-31,1.72,-0.494559
2023-11-30,2.02,-1.147119
2023-12-31,2.03,-0.384429


In [42]:
ENSO_index.tail()

,SST,SOI
2025-03-31,0.01,1.338456
2025-04-30,-0.14,0.559077
2025-05-31,-0.13,0.803407
2025-06-30,-0.06,0.555814
2025-07-31,-0.06,0.778498


### selects only the columns we actually need 

In [43]:
ENSO_index = ENSO_index.loc[:,['SST','SOI']]

### divides the SOI by 10 to get something closer to the NIWA SOI 

In [44]:
# ENSO_index.loc[:,'SOI'] /= 10.

In [45]:
ENSO_index.head()

,SST,SOI
2023-08-31,1.35,-1.013194
2023-09-30,1.60,-1.321736
2023-10-31,1.72,-0.494559
2023-11-30,2.02,-1.147119
2023-12-31,2.03,-0.384429


In [46]:
ENSO_index.tail()

,SST,SOI
2025-03-31,0.01,1.338456
2025-04-30,-0.14,0.559077
2025-05-31,-0.13,0.803407
2025-06-30,-0.06,0.555814
2025-07-31,-0.06,0.778498


### get the last date 

In [47]:
last_date = pd.to_datetime(ENSO_index.index[-1])

In [48]:
last_date

Timestamp('2025-07-31 00:00:00')

### get the values and the basis for plotting

In [49]:
soi_values = ENSO_index.SOI.values

In [50]:
nino_values = ENSO_index.SST.values 

In [51]:
basis = np.arange(len(ENSO_index))

### build the list of dates for X-axis labeling 

In [52]:
dates = ENSO_index.index.to_pydatetime()

In [53]:
dates_str = [f"{x:%b %Y}" for x in dates]

### Now read the ENSO categories from NOAA, BoM, APCC and NOAA

In [54]:
enso_cat_int = read_ENSO_categories()

/tmp/ipykernel_118113/1927798126.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enso_cat_int = enso_cat.replace(cat_to_int)
/tmp/ipykernel_118113/1927798126.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2019-01' '2019-02' '2019-03' '2019-04' '2019-05' '2019-06' '2019-07'
 '2019-08' '2019-09' '2019-10' '2019-11' '2019-12' '2020-01' '2020-02'
 '2020-03' '2020-04' '2020-05' '2020-06' '2020-07' '2020-08' '2020-09'
 '2020-10' '2020-11' '2020-12' '2021-01' '2021-02' '2021-03' '2021-04'
 '2021-05' '2021-06' '2021-07' '2021-08' '2021-09' '2021-10' '2021-11'
 '2021-12' '2022-01' '2022-02' '2022-03' '2022-04' '2022-05' '2022-06'
 '2022-07' '2022-08' '2022-09' '2022-10' '2022-11' '20

In [55]:
ENSO_index.tail()

,SST,SOI
2025-03-31,0.01,1.338456
2025-04-30,-0.14,0.559077
2025-05-31,-0.13,0.803407
2025-06-30,-0.06,0.555814
2025-07-31,-0.06,0.778498


In [56]:
enso_cat_int.tail()

,NOAA,BoM,APCC,NIWA
date,,,,
2025-03-31,-3,0,0,0
2025-04-30,0,0,0,0
2025-05-31,0,0,0,0
2025-06-30,0,0,0,0
2025-07-31,0,0,0,0


In [57]:
# ENSO_index.index = ENSO_index.index.floor('D')

In [58]:
enso_cat_int = enso_cat_int.loc[ENSO_index.index,:]

### colors for the heatmap 

In [59]:
col_dict={-3:"#4233ff",
          -2:"#33a2ff",
          -1:"#33ffbb",
          0:"#d4d4d4",
          1:"#f3ff33", 
          2:"#ffa233",
          3:"#dc0f09"
         }

cm = ListedColormap([col_dict[x] for x in col_dict.keys()])

In [60]:
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 

In [61]:
f = plt.figure(figsize=(8,8))

# defines the axes [left, bottom, width, height]

ax0 = f.add_axes([0.1, 0.66, 0.9, 0.3])

ax0.set_title("Pacific Regional Climate Centre ENSO tracker\n", fontsize=16)

ax1 = f.add_axes([0.1, 0.33, 0.755, 0.33])

ax2 = f.add_axes([0.1, 0.05, 0.755, 0.30])

#============================================================================================================================
# first set of axes: ENSO categories heatmap 

sns.heatmap(enso_cat_int.T, cmap=cm, center=0, vmin=-3, vmax=3, ax=ax0, \
            cbar_kws={'aspect':5, 'pad':0.01}) 

colorbar = ax0.collections[0].colorbar 

# ax0.set_yticks([0.8, 1.8, 2.8, 3.8]);
ax0.set_yticks([0.6, 1.6, 2.6, 3.6]);

ax0.set_yticklabels(['NOAA','BoM','APCC','NIWA'], fontsize=14)

ax0.set_ylim(bottom=0,top=4)

[ax0.axvline(x, color='k', lw=2, alpha=0.5) for x in range(0,len(enso_cat_int) + 1)];

[ax0.axhline(x, color='k', lw=2, alpha=0.5) for x in range(1,5)];

ax0.tick_params(direction='out', length=0, width=0)

colorbar.set_ticks([-2.5, -1.75, -0.8,  0.,  0.90, 1.75,  2.6])

colorbar.set_ticklabels(["La Niña", "La Niña Alert", "La Niña Watch", "Neutral", "El Niño Watch", "El Niño Alert", "El Niño"])    

colorbar.ax.tick_params(direction='out', length=0, width=0)

#============================================================================================================================
# second set of axes: ENSO barplot

ax1.bar(basis[nino_values>=0],nino_values[nino_values>=0], width=1, facecolor='coral', alpha=.8, edgecolor='k', lw=2)
ax1.bar(basis[nino_values<0],nino_values[nino_values<0], width=1, facecolor='steelblue', alpha=.8, edgecolor='k', lw=2)

ax1.set_xticks(basis)

ax1.set_xticklabels(dates_str, rotation=90)

ax1.set_xlim([-0.5,len(dates) - 0.5])

# get the minimum and maximum values, and set limits and ticks for the y-axis accordingly 
max_nino = np.ceil(np.max(np.abs(nino_values)))
min_nino =- max_nino

if max_nino > 1: 
    ax1.set_ylim([min_nino, max_nino])
    ax1.set_yticks(np.arange(min_nino+0.5, max_nino, 0.5))
else: 
    ax1.set_ylim([-1, 1])
    ax1.set_yticks([-0.7, 0, 0.7])                  
    
ax1.axhline(-0.7, ls=':', color='b')
ax1.axhline(0.7, ls=':', color='r')

ax1.yaxis.tick_right()

ax1.set_ylabel('monthly NINO 3.4\nanomalies (℃)', fontsize=14)

ax1.yaxis.set_label_position("right")

[ax1.axvline(x, color='k', lw=1, alpha=0.5, ls=':') for x in np.arange(0.5,len(dates)-0.5)];

ax1.set_xlabel('month', fontsize=14)

ax1.text(0.0125, 0.9, 'NINO 3.4 (from NOAA CPC)', transform=ax1.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=1, edgecolor='white'))

#============================================================================================================================
# second set of axes: SOI barplot

ax2.bar(basis[soi_values>=0],soi_values[soi_values>=0], width=1, facecolor='steelblue', alpha=.8, edgecolor='k', lw=2)
ax2.bar(basis[soi_values<0],soi_values[soi_values<0], width=1, facecolor='coral', alpha=.8, edgecolor='k', lw=2)

ax2.set_xticks(basis)

ax2.set_xticklabels(dates_str, rotation=90)

ax2.set_xlim([-0.5,len(dates) - 0.5])

# get the minimum and maximum values, and set limits and ticks for the y-axis accordingly 
max_soi = np.ceil(np.max(np.abs(soi_values)))
min_soi =- max_soi

if max_soi > 1.5: 
    ax2.set_ylim([min_soi, max_soi])
    ax2.set_yticks(np.arange(min_soi+0.5, max_soi, 0.5))
else: 
    ax2.set_ylim([-1.5, 1.5])
    ax2.set_yticks([-1, 0, 1])       

ax2.axhline(-1, ls=':', color='r')
ax2.axhline(1, ls=':', color='b')

ax2.yaxis.tick_right()

ax2.yaxis.set_label_position("right")

ax2.set_ylabel('monthly SOI (std)', fontsize=14)

ax2.yaxis.set_label_position("right")

ax2.text(0.0125, 0.9, 'SOI (from NOAA CPC)', transform=ax2.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=1, edgecolor='white'))

[ax2.axvline(x, color='k', lw=1, alpha=0.5, ls=':') for x in np.arange(0.5,len(dates)-0.5)];

# f.patch.set_facecolor('0.99')
f.patch.set_facecolor('w')

SyntaxError: unterminated string literal (detected at line 36) (3418819755.py, line 36)

### saves to file 

In [ ]:
f.savefig('../figures/prototype_Pacific_RCC.png', dpi=200, bbox_inches='tight')

### push to github

In [ ]:
%cd "/home/fauchereaun/operational/PI-RCC/Pacific_RCC" 

In [ ]:
# Specify the Git commands as strings
git_add = 'git add figures/prototype_Pacific_RCC.png notebooks/Pacific_RCC_ENSO_tracker.ipynb'
git_commit = f'git commit -m "updated on {year}-{month}"'
git_push = 'git push'

In [ ]:
!{git_add}

In [ ]:
!{git_commit}

In [ ]:
!{git_push}